In [1]:
## ANN chiller group model
import pandas as pd

# Load the data
data = pd.read_csv('ElectricalGroupResults.csv')

# Display the first few rows of the dataset
print(data.shape)
data.drop_duplicates(subset = ['Time'], inplace = True)
print(data.shape)
data.reset_index(drop = True, inplace = True)
data.head()


(8761, 12)
(8761, 12)


,Time,electricalGroup.Ppv,electricalGroup.Ir,electricalGroup.Pdem,electricalGroup.Ppla,electricalGroup.Pgen,electricalGroup.Soc,electricalGroup.Pbat,electricalGroup.Pgri.real,electricalGroup.batCon1.chaCyc.active,electricalGroup.batCon1.staBy.active,electricalGroup.batCon1.disCyc.active
0,0,0.0,0.0,-10994957.0,0.0,7196211.0,0.50,0.0,3798746.50,0.0,0.0,1.0
1,3600,0.0,0.0,-10955582.0,0.0,7400128.0,0.49,-3557542.0,-2087.75,0.0,0.0,1.0
2,7200,0.0,0.0,-10891327.0,0.0,7633626.0,0.49,-3260256.0,-2555.10,0.0,0.0,1.0
3,10800,0.0,0.0,-10986413.0,0.0,7851213.5,0.48,-3136251.2,-1051.21,0.0,0.0,1.0
4,14400,0.0,0.0,-11105312.0,0.0,8088488.5,0.47,-3017839.8,-1015.82,0.0,0.0,1.0


In [2]:
# Outlier clipping
inputs = ['electricalGroup.Ir', 'electricalGroup.Pdem', 'electricalGroup.Ppla', 'electricalGroup.Pgen'] ##States and actions
outputs = ['electricalGroup.Ppv', 'electricalGroup.Soc', 'electricalGroup.Pbat', 'electricalGroup.Pgri.real'] 

columns = inputs + outputs

mean = data[columns].mean()
q1 = data[columns].quantile(0.25)
q3 = data[columns].quantile(0.75)
iqr = q3 - q1
data[columns] = data[columns].clip(q1 - 1.5*iqr, q3 + 1.5*iqr, axis = 1)
data.head()

,Time,electricalGroup.Ppv,electricalGroup.Ir,electricalGroup.Pdem,electricalGroup.Ppla,electricalGroup.Pgen,electricalGroup.Soc,electricalGroup.Pbat,electricalGroup.Pgri.real,electricalGroup.batCon1.chaCyc.active,electricalGroup.batCon1.staBy.active,electricalGroup.batCon1.disCyc.active
0,0,0.0,0.0,-10994957.0,0.0,7196211.0,0.25,0.0,3798746.50,0.0,0.0,1.0
1,3600,0.0,0.0,-10955582.0,0.0,7400128.0,0.25,-1571770.8,-2087.75,0.0,0.0,1.0
2,7200,0.0,0.0,-10891327.0,0.0,7633626.0,0.25,-1571770.8,-2555.10,0.0,0.0,1.0
3,10800,0.0,0.0,-10986413.0,0.0,7851213.5,0.25,-1571770.8,-1051.21,0.0,0.0,1.0
4,14400,0.0,0.0,-11105312.0,0.0,8088488.5,0.25,-1571770.8,-1015.82,0.0,0.0,1.0


In [3]:
# Define inputs and outputs
##ANN outputs

# Create input and output datasets
X = pd.concat([data[inputs].iloc[1:].reset_index(drop = True), data[outputs].iloc[:-1]], axis=1)
y = data[outputs].iloc[1:]

X.index = data.Time[1:]
y.index = data.Time[1:]
# X = X.iloc[1:]
# X = pd.concat([X, y.iloc[:-1]], axis=1, ignore_index=True)

In [4]:
X.head()

,electricalGroup.Ir,electricalGroup.Pdem,electricalGroup.Ppla,electricalGroup.Pgen,electricalGroup.Ppv,electricalGroup.Soc,electricalGroup.Pbat,electricalGroup.Pgri.real
Time,,,,,,,,
3600,0.0,-10955582.0,0.0,7400128.0,0.0,0.25,0.0,3798746.50
7200,0.0,-10891327.0,0.0,7633626.0,0.0,0.25,-1571770.8,-2087.75
10800,0.0,-10986413.0,0.0,7851213.5,0.0,0.25,-1571770.8,-2555.10
14400,0.0,-11105312.0,0.0,8088488.5,0.0,0.25,-1571770.8,-1051.21
18000,0.0,-11151833.0,0.0,8273101.5,0.0,0.25,-1571770.8,-1015.82


In [5]:
y.head()

,electricalGroup.Ppv,electricalGroup.Soc,electricalGroup.Pbat,electricalGroup.Pgri.real
Time,,,,
3600,0.0,0.25,-1571770.8,-2087.75
7200,0.0,0.25,-1571770.8,-2555.10
10800,0.0,0.25,-1571770.8,-1051.21
14400,0.0,0.25,-1571770.8,-1015.82
18000,0.0,0.25,-1571770.8,-1185.01


In [6]:
X.shape

(8760, 8)

In [7]:
y.shape

(8760, 4)

In [8]:
index = pd.Series(X.index)
index[(index >= 2*30*86400) & (index<=3*30*86400)]

1439    5184000
1440    5187600
1441    5191200
1442    5194800
1443    5198400
         ...   
2155    7761600
2156    7765200
2157    7768800
2158    7772400
2159    7776000
Name: Time, Length: 721, dtype: int64

In [9]:
month = 30*86400
import numpy as np
def split_data_20_10(X, y):
    X_train, X_test, y_train, y_test = [], [], [], []
    index = pd.Series(X.index)
    for i in range(0, int(X.index[-1])//month):

        index_filter = index[(index >= i*30*86400) & (index<=(i+1)*30*86400)]
        
        X_train.append(X.loc[index_filter[:int(len(index_filter)*0.7)]])
        y_train.append(y.loc[index_filter[:int(len(index_filter)*0.7)]])
        X_test.append(X.loc[index_filter[int(len(index_filter)*0.7):]])
        y_test.append(y.loc[index_filter[int(len(index_filter)*0.7):]])

    return np.vstack(X_train), np.vstack(X_test), np.vstack(y_train), np.vstack(y_test)

In [10]:
X_train, X_test, y_train, y_test = split_data_20_10(X, y)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Split the data
# X_train, X_test, y_train, y_test = X.iloc[:int(0.8*len(X))], X.iloc[int(0.8*len(X)):], y.iloc[:int(0.8*len(y))], y.iloc[int(0.8*len(y)):]

# Normalize the features
scaler_X = MinMaxScaler().fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

scaler_y = MinMaxScaler().fit(y_train)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)



In [12]:
X_train.shape

(6047, 8)

In [15]:
X_train.shape[1]

8

In [13]:
import tensorflow as tf

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='sigmoid', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(len(outputs)),  # Output layer with the same number of neurons as target features
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss='mean_squared_error', metrics=['mae']) 


In [14]:
model.load_weights("trained_model_Electrical_ANN.h5")

ValueError: Cannot assign value to variable ' dense/kernel:0': Shape mismatch.The variable shape (8, 128), and the assigned value shape (11, 128) are incompatible.

In [ ]:
history = model.fit(X_train, y_train, epochs=800, batch_size=256, validation_split=0.2)

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


In [ ]:
##Visualization:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

# Plot training & validation mean absolute error values
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend(['Train', 'Validation'])

plt.show()


In [ ]:
# Save the model
model.save('trained_model_Electrical_ANN.h5')

# Load the model (if needed)
loaded_model = tf.keras.models.load_model('trained_model_Electrical_ANN.h5')


In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Convert predictions and actual values to DataFrames for easier manipulation
predictions_df = pd.DataFrame(scaler_y.inverse_transform(y_pred), columns=outputs)
actuals_df = pd.DataFrame(scaler_y.inverse_transform(y_test), columns=outputs)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_metrics(actuals, predictions):
    metrics = {}
    for output in actuals.columns:
        actual_values = actuals[output]
        predicted_values = predictions[output]
        
        # Calculate metrics
        mse = mean_squared_error(actual_values, predicted_values)
        rmse = np.sqrt(mse)
        mean_actual = np.mean(actual_values)
        
        # CVRMSE (Coefficient of Variation of RMSE)
        cvrmse = (rmse / mean_actual) * 100
        
        # NMBE (Normalized Mean Bias Error)
        nmbe = (np.mean(predicted_values - actual_values) / mean_actual) * 100
        
        # R2 (Coefficient of Determination)
        r2 = r2_score(actual_values, predicted_values)
        
        metrics[output] = {'CVRMSE': cvrmse, 'NMBE': nmbe, 'R2': r2}
    return metrics

def plot_actual_vs_predicted(actuals, predictions, output_names):
    metrics = calculate_metrics(actuals, predictions)
    
    num_outputs = len(output_names)
    plt.figure(figsize=(15, num_outputs * 5))
    
    for i, output in enumerate(output_names):
        plt.subplot(num_outputs, 1, i + 1)
        plt.plot(actuals.index, actuals[output], label='Actual', color='blue', linestyle='--')
        plt.plot(predictions.index, predictions[output], label='Predicted', color='red', linestyle='-')
        
        # Retrieve metrics for current output
        cvrmse = metrics[output]['CVRMSE']
        nmbe = metrics[output]['NMBE']
        r2 = metrics[output]['R2']
        
        # Print metrics on plot
        plt.title(f'Actual vs. Predicted for {output}')
        plt.xlabel('Sample Index')
        plt.ylabel(output)
        plt.legend()
        plt.text(0.05, 0.95, f'CVRMSE: {cvrmse:.2f}%', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
        plt.text(0.05, 0.90, f'NMBE: {nmbe:.2f}%', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
        plt.text(0.05, 0.85, f'R2: {r2:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

# Call the updated plotting function
plot_actual_vs_predicted(actuals_df, predictions_df, outputs)
